High Value Customer Identification (Insiders)

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import requests

In [2]:
!pip list --format=freeze > requirements.txt

In [3]:
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their␣
,→names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""

In [4]:
soup = BeautifulSoup( html_doc, 'html.parser' )

In [5]:
soup.find_all("a", id = "link1")[0].string

'Elsie'

In [6]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text, "html.parser")

In [7]:
products = soup.find( "ul" , class_='products-listing small')

In [8]:
product_list = products.find_all( "article", class_="hm-product-item")

product_id = [ p.get( 'data-articlecode') for p in product_list]

product_category = [ p.get( 'data-category') for p in product_list]

list_name = products.find_all( "a", class_ = 'link')

product_name = [ p.get_text() for p in list_name]

price_list = products.find_all( "span", class_ = 'price regular')

product_price = [ p.get_text() for p in price_list]

In [9]:
data = pd.DataFrame( [product_id, product_category, product_name, product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']
data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )

data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:39
1,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-29 16:18:39
2,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-11-29 16:18:39
3,0690449067,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:39
4,1008549006,men_jeans_regular,Regular Jeans,$ 24.99,2022-11-29 16:18:39


In [10]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

In [11]:
soup = BeautifulSoup( page.text, "html.parser")

total_item = soup.find_all( 'h2', class_='load-more-heading')[0].get( 'data-total')
total_item

'90'

In [12]:
pages_number = np.ceil(int(total_item) / 36 )
pages_number

3.0

In [13]:
url02 = url + "?page-size=" + str( int( pages_number*36 ) )
url02

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=108'

# Individual product collection

In [14]:
url = 'https://www2.hm.com/en_us/productpage.0985159008.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text , 'html.parser' )

soup

<!DOCTYPE HTML>

<html class="no-js en-us" lang="en-US">
<head>
<link href="https://s1-cdn.hm.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://s1-cdn.hm.com" rel="preconnect"/>
<link href="https://tags.tiqcdn.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://tags.tiqcdn.com" rel="preconnect"/>
<link href="https://lp2.hm.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://lp2.hm.com" rel="preconnect"/>
<link href="https://cdn-pci.optimizely.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://cdn-pci.optimizely.com" rel="preconnect"/>
<link href="https://s.go-mpulse.net" rel="dns-prefetch"/>
<link crossorigin="" href="https://s.go-mpulse.net" rel="preconnect"/>
<script defer="" src="https://cdn.cookielaw.org/consent/511a7faf-cc2d-4b69-86fb-a37761829422/OtAutoBlock.js" type="text/javascript"></script>
<script charset="UTF-8" data-document-language="true" data-domain-script="511a7faf-cc2d-4b69-86fb-a37761829422" defer="" src="https://cdn.cookielaw.o

In [15]:

product_list = soup.find_all( 'li' , class_ = 'list-item' )
product_list

[<li class="list-item hidden">
 <a aria-checked="false" class="filter-option miniature" data-articlecode="0985159001" data-color="Black" data-sizes="" href="/en_us/productpage.0985159001.html" id="filter-colour-0985159001" role="radio" title="Black">
 <noscript data-alt="Black" data-src="//lp2.hm.com/hmgoepprod?set=quality%5B79%5D%2Csource%5B%2F15%2F58%2F155836e4c168f8ed426eede291e4107f87831490.jpg%5D%2Corigin%5Bdam%5D%2Ccategory%5B%5D%2Ctype%5BDESCRIPTIVESTILLLIFE%5D%2Cres%5Bm%5D%2Chmver%5B2%5D&amp;call=url[file:/product/miniature]">
 <img alt="Black" src="//lp2.hm.com/hmgoepprod?set=quality%5B79%5D%2Csource%5B%2F15%2F58%2F155836e4c168f8ed426eede291e4107f87831490.jpg%5D%2Corigin%5Bdam%5D%2Ccategory%5B%5D%2Ctype%5BDESCRIPTIVESTILLLIFE%5D%2Cres%5Bm%5D%2Chmver%5B2%5D&amp;call=url[file:/product/miniature]"/>
 </noscript>
 <span></span>
 </a>
 </li>,
 <li class="list-item hidden">
 <a aria-checked="false" class="filter-option miniature" data-articlecode="0985159002" data-color="Denim blue"

In [16]:
url = 'https://www2.hm.com/en_us/productpage.0985159007.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text , 'html.parser' )

soup

<!DOCTYPE HTML>

<html class="no-js en-us" lang="en-US">
<head>
<link href="https://s1-cdn.hm.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://s1-cdn.hm.com" rel="preconnect"/>
<link href="https://tags.tiqcdn.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://tags.tiqcdn.com" rel="preconnect"/>
<link href="https://lp2.hm.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://lp2.hm.com" rel="preconnect"/>
<link href="https://cdn-pci.optimizely.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://cdn-pci.optimizely.com" rel="preconnect"/>
<link href="https://s.go-mpulse.net" rel="dns-prefetch"/>
<link crossorigin="" href="https://s.go-mpulse.net" rel="preconnect"/>
<script defer="" src="https://cdn.cookielaw.org/consent/511a7faf-cc2d-4b69-86fb-a37761829422/OtAutoBlock.js" type="text/javascript"></script>
<script charset="UTF-8" data-document-language="true" data-domain-script="511a7faf-cc2d-4b69-86fb-a37761829422" defer="" src="https://cdn.cookielaw.o

In [17]:
product_list = soup.find_all( 'a' , class_ = 'filter-option miniature' )

# product id list
product_ids = [p.get( 'data-articlecode' ) for p in product_list]

# product color list
product_colors = [p.get( 'data-color' ) for p in product_list]

df_color = pd.DataFrame({"id" : product_ids, "color" : product_colors })
df_color.head(10)

,id,color
0,0985159001,Black
1,0985159002,Denim blue
2,0985159003,Dark gray
3,0985159004,Light denim blue
4,0985159005,Dark blue
5,0985159006,Dark denim blue
6,0985159008,Dark gray


In [18]:
url = 'https://www2.hm.com/en_us/productpage.0985159007.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text , 'html.parser' )

product_list = soup.find_all('div', class_='details-attributes-list-item')


# remove empty strings
product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_list]


# list to dataframe
data_composition = pd.DataFrame(product_composition).T


# data aux
data_composition_aux = data_composition.iloc[0]


# rename df columns
data_composition.columns = data_composition_aux


# forward fillna
data_composition = data_composition.iloc[1:].fillna(method='ffill')


# drop duplicates
data_composition = data_composition.drop_duplicates(subset=['Fit', 'Composition', 'Art. No.'], keep='first', inplace=False, ignore_index=False)


# only selected columns
data_composition = data_composition[['Fit', 'Composition', 'Art. No.']].copy()

data_composition.columns = data_composition.columns.str.lower()

data_composition['style_id'] = data_composition['art. no.'].str[:-3]

data_composition['color_id'] = data_composition['art. no.'].str[-3:]


# Produto Final
data_composition

,fit,composition,art. no.,style_id,color_id
1,Skinny fit,"Shell: Cotton 99%, Spandex 1%",0985159007,0985159,007
2,Skinny fit,"Pocket lining: Polyester 65%, Cotton 35%",0985159007,0985159,007


# Multiple products

In [6]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text, "html.parser")

total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
total_item

'90'

In [7]:
page_number = np.round( int( total_item ) / 36 )
page_number

2.0

In [8]:
urlfull = url + '?page-size=' + str( int( page_number*36 ) )
urlfull

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=72'

In [ ]:
urlfull = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text, "html.parser")

total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
total_item

In [20]:
product_list = products.find_all( "article", class_="hm-product-item")

product_id = [ p.get( 'data-articlecode') for p in product_list]

product_category = [ p.get( 'data-category') for p in product_list]

list_name = products.find_all( "a", class_ = 'link')

product_name = [ p.get_text() for p in list_name]

price_list = products.find_all( "span", class_ = 'price regular')

product_price = [ p.get_text() for p in price_list]

In [21]:
df_products = pd.DataFrame( [product_id, product_category, product_name, product_price] ).T
df_products.columns = ['product_id', 'product_category', 'product_name', 'product_price']
df_products['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )
#df_products['style_id'] = df_products['product_id'].str[:-3]
#df_products['color_id'] = df_products['product_id'].str[-3:]
df_products.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45
1,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-29 16:18:45
2,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-11-29 16:18:45
3,0690449067,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45
4,1008549006,men_jeans_regular,Regular Jeans,$ 24.99,2022-11-29 16:18:45


In [22]:
headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

df_details = pd.DataFrame()

#unique columns for all products
aux = []

cols = ['Additional material information','Art. No.','Care instructions','Collection','Composition','Concept','Description','Fit','Imported','Material','Size']
df_pattern = pd.DataFrame( columns = cols)

for i in df_products['product_id'].unique():

    url = f'https://www2.hm.com/en_us/productpage.{i}.html'

    page = requests.get( url, headers=headers )

    soup = BeautifulSoup( page.text , 'html.parser' )

    product_list = soup.find_all('div', class_='details-attributes-list-item')


    # remove empty strings
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_list]


    # list to dataframe
    data_composition = pd.DataFrame(product_composition).T


    # data aux
    data_composition_aux = data_composition.iloc[0]


    # rename df columns
    data_composition.columns = data_composition_aux

    aux = aux + data_composition.columns.tolist()

    aux = [i for i in aux if "messages." not in i]

    # forward fillna
    data_composition = data_composition.iloc[1:].fillna(method='ffill')

    data_composition = pd.concat( [df_pattern, data_composition], axis=0)


    # drop duplicates
    data_composition = data_composition.drop_duplicates(subset=['Fit', 'Composition', 'Art. No.'], keep='first', inplace=False, ignore_index=False)


    # only selected columns
    data_composition = data_composition[cols].copy()

    data_composition.columns = data_composition.columns.str.lower()

    data_composition['style_id'] = data_composition['art. no.'].str[:-3]

    data_composition['color_id'] = data_composition['art. no.'].str[-3:]



    # Produto Final
    df_details = pd.concat( [df_details, data_composition] , axis=0)

In [23]:
df_details.head(3)

,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,size,style_id,color_id
1,NaN,0690449043,Only non-chlorine bleach when needed,NaN,"Cotton 98%, Spandex 2%",DENIM,Light denim blue,Skinny fit,Yes,keyFibreType,NaN,0690449,043
2,NaN,0690449043,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Light denim blue,Skinny fit,Yes,Denim,NaN,0690449,043
1,NaN,1024256001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Slim fit,Yes,Denim,NaN,1024256,001


In [24]:
df_products2 = df_products.merge(df_details, left_on='product_id', right_on='art. no.', how='left', validate='one_to_many')
print( df_products.shape, df_products2.shape)
df_products2.head(3)

(36, 5) (67, 18)


,product_id,product_category,product_name,product_price,scrapy_datetime,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,size,style_id,color_id
0,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45,NaN,0690449043,Only non-chlorine bleach when needed,NaN,"Cotton 98%, Spandex 2%",DENIM,Light denim blue,Skinny fit,Yes,keyFibreType,NaN,0690449,043
1,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45,NaN,0690449043,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Light denim blue,Skinny fit,Yes,Denim,NaN,0690449,043
2,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-29 16:18:45,NaN,1024256001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Slim fit,Yes,Denim,NaN,1024256,001


In [25]:
df_products2.to_csv("df_products.csv", index=False)